In [1]:
!pip install Flask

In [14]:
def generate_fruit_recommendations(go_out_to_party, flavor_preference, texture_dislike, price_range):
    prompt = "Given the following rules:\n\n"
    
    # Add rules based on user responses
    if go_out_to_party.lower() == "yes":
        allowed_fruits = ["apples", "pears", "grapes", "watermelon"]
        prompt += "If you go out to party on weekends, the allowed fruits are: " + ", ".join(allowed_fruits) + ".\n"
    else:
        allowed_fruits = ["oranges", "apples", "pears", "grapes", "watermelon", "lemon", "lime"]
    
    if flavor_preference.lower() == "cider":
        prompt += "If you like cider, show apples, oranges, lemon, lime.\n"
        allowed_fruits = ["apples", "oranges", "lemon", "lime"]
    elif flavor_preference.lower() == "sweet":
        prompt += "If you like sweet, show watermelon, orange.\n"
        allowed_fruits = ["watermelon", "orange"]
    elif flavor_preference.lower() == "waterlike":
        prompt += "If you like waterlike, show watermelon.\n"
        allowed_fruits = ["watermelon"]
    
    if "watermelon" in allowed_fruits and "grapes" in allowed_fruits:
        prompt += "If grapes are chosen, remove watermelon from the list.\n"
        allowed_fruits.remove("watermelon")
    
    if texture_dislike.lower() == "smooth" and "pears" in allowed_fruits:
        prompt += "If you don't like smooth texture, remove pears.\n"
        allowed_fruits.remove("pears")
    elif texture_dislike.lower() == "slimy":
        prompt += "If you don't like slimy texture, remove watermelon, lime, and grape.\n"
        for fruit in ["watermelon", "lime", "grape"]:
            if fruit in allowed_fruits:
                allowed_fruits.remove(fruit)
    elif texture_dislike.lower() == "waterlike" and "watermelon" in allowed_fruits:
        prompt += "If you don't like waterlike texture, remove watermelon.\n"
        allowed_fruits.remove("watermelon")
    
    if price_range < 3:
        prompt += "If price is less than $3, remove lime and watermelon.\n"
        for fruit in ["lime", "watermelon"]:
            if fruit in allowed_fruits:
                allowed_fruits.remove(fruit)
    elif price_range > 4 and price_range < 7:
        prompt += "If price is greater than $4 and less than $7, remove pears and apples.\n"
        for fruit in ["pears", "apples"]:
            if fruit in allowed_fruits:
                allowed_fruits.remove(fruit)
    
    prompt += "\nBased on your preferences, the recommended fruits are: " + ", ".join(allowed_fruits) + ".\n"
    
    return prompt

# Updated function to get user input
def get_user_input():
    go_out_to_party = input("Do you go out to party on weekends? (yes or no): ")
    flavor_preference = input("What flavors do you like? (cider, sweet, waterlike): ")
    texture_dislike = input("What texture do you dislike? (smooth, slimy, rough): ")
    print("What price range will you buy drinks for? ($1, $2, $3, $4, $5, $6, $7, $8, $9, $10): ")
    price_range_input = input("Please enter the corresponding number: ")
    try:
        price_range = int(price_range_input)
        if price_range < 1 or price_range > 10:
            raise ValueError("Price range must be between 1 and 10")
        return go_out_to_party, flavor_preference, texture_dislike, price_range
    except ValueError as e:
        print("Invalid input:", e)
        return None

# Get user input with error handling
user_input = None
while user_input is None:
    user_input = get_user_input()

# Generate recommendations based on user input
go_out_to_party, flavor_preference, texture_dislike, price_range = user_input
recommendations_prompt = generate_fruit_recommendations(go_out_to_party, flavor_preference, texture_dislike, price_range)
print(recommendations_prompt)


Do you go out to party on weekends? (yes or no): yes
What flavors do you like? (cider, sweet, waterlike): sweet
What texture do you dislike? (smooth, slimy, rough): smooth
What price range will you buy drinks for? ($1, $2, $3, $4, $5, $6, $7, $8, $9, $10): 
Please enter the corresponding number: 5
Given the following rules:

If you go out to party on weekends, the allowed fruits are: apples, pears, grapes, watermelon.
If you like sweet, show watermelon, orange.
If price is greater than $4 and less than $7, remove pears and apples.

Based on your preferences, the recommended fruits are: watermelon, orange.



In [25]:
# Flask API Setup
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/recommend_fruits', methods=['POST'])
def recommend_fruits():
    data = request.json
    go_out_to_party = data.get("go_out_to_party")
    flavor_preference = data.get("flavor_preference")
    texture_dislike = data.get("texture_dislike")
    price_range = data.get("price_range")

    if None in [go_out_to_party, flavor_preference, texture_dislike, price_range]:
        return jsonify({"error": "Missing required fields"}), 400
    
    recommendations_prompt = generate_fruit_recommendations(go_out_to_party, flavor_preference, texture_dislike, price_range)
    return jsonify({"recommendations": recommendations_prompt})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1